# 멀티 프로세스와 멀티 스레드

멀티 프로세스 : subprocess 모듈을 사용한다.  
멀티 스레드 : threading 모듈을 사용한다.

## subprocess 모듈
유닉스 계열에서만 실행된다.

In [1]:
import subprocess
subprocess.run(['echo', '이것은 subprocess입니다.'])

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [2]:
subprocess.run(['sleep', '10'])

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

## threading 모듈

In [3]:
import queue
import threading

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()
        if item is None:
            break
        #작업을 처리한다.
        print('스레드 {0} : 처리 완료 {1}'.format(num+1, item))
        q.task_done()

In [ ]:
num_worker_threads = 5
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=worker, args=(i,))
    t.start()
    threads.append(t)

for item in range(20):
    q.put(item)
    
q.join()

for i in range(num_worker_threads):
    q.put(None)
for t in threads:
    t.join()

스레드 1 : 처리 완료 0스레드 5 : 처리 완료 1
스레드 1 : 처리 완료 2
스레드 1 : 처리 완료 3
스레드 3 : 처리 완료 4
스레드 3 : 처리 완료 5
스레드 3 : 처리 완료 6
스레드 3 : 처리 완료 7
스레드 4 : 처리 완료 8
스레드 4 : 처리 완료 9
스레드 4 : 처리 완료 10
스레드 4 : 처리 완료 11
스레드 4 : 처리 완료 12
스레드 4 : 처리 완료 13
스레드 3 : 처리 완료 14
스레드 3 : 처리 완료 15
스레드 3 : 처리 완료 16
스레드 3 : 처리 완료 17
스레드 3 : 처리 완료 18
스레드 3 : 처리 완료 19



## 뮤텍스와 세마포어

In [1]:
from threading import Thread, Lock
import threading

def worker(mutex, data, thread_safe):
    if thread_safe:
        mutex.acquire()
    try:
        print('스레드 {0} : {1}\n'.format(threading.get_ident(), data))
    finally:
        if thread_safe:
            mutex.release()

In [2]:
threads = []
thread_safe = False
mutex = Lock()
for i in range(20):
    t = Thread(target=worker, args=(mutex, i, thread_safe))
    t.start()
    threads.append(t)
for t in threads:
    t.join()

스레드 9196 : 0

스레드 11100 : 1

스레드 13676 : 2

스레드 1296 : 3

스레드 9128 : 4

스레드 9520 : 5

스레드 3660 : 6

스레드 14476 : 7

스레드 11768 : 8

스레드 10536 : 9

스레드 15908 : 10

스레드 16400 : 11

스레드 15316 : 12

스레드 13168 : 13

스레드 16432 : 14

스레드 5708 : 15

스레드 16096 : 16

스레드 2488 : 17

스레드 16424 : 18

스레드 11480 : 19



In [3]:
threads = []
thread_safe = True
mutex = Lock()
for i in range(20):
    t = Thread(target=worker, args=(mutex, i, thread_safe))
    t.start()
    threads.append(t)
for t in threads:
    t.join()

스레드 16436 : 0

스레드 16396 : 1

스레드 4016 : 2

스레드 9140 : 3

스레드 15236 : 4

스레드 4480 : 5

스레드 12920 : 6

스레드 10636 : 7

스레드 1600 : 8

스레드 13552 : 9

스레드 10736 : 10

스레드 12736 : 11

스레드 12092 : 12

스레드 16508 : 13

스레드 14556 : 14

스레드 12512 : 15

스레드 6776 : 16

스레드 11448 : 17

스레드 18312 : 18

스레드 3828 : 19



In [5]:
import threading
import time

class ThreadPool(object):
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()
        
    def acquire(self, name):
        with self.lock:
            self.active.append(name)
            print('획득: {0} | 스레드 풀: {1}'.format(name, self.active))
            
    def release(self, name):
        with self.lock:
            self.active.remove(name)
            print('반환: {0} | 스레드 풀: {1}'.format(name, self.active))
            
def worker(semaphore, pool):
    with semaphore:
        name = threading.currentThread().getName()
        pool.acquire(name)
        time.sleep(1)
        pool.release(name)

In [6]:
threads = []
pool = ThreadPool()
semaphore = threading.Semaphore(3)
for i in range(10):
    t = threading.Thread(target=worker, name="스레드" + str(i), args=(semaphore, pool))
    t.start()
    threads.append(t)
for t in threads:
    t.join()

획득: 스레드0 | 스레드 풀: ['스레드0']
획득: 스레드1 | 스레드 풀: ['스레드0', '스레드1']
획득: 스레드2 | 스레드 풀: ['스레드0', '스레드1', '스레드2']
반환: 스레드0 | 스레드 풀: ['스레드1', '스레드2']
획득: 스레드3 | 스레드 풀: ['스레드1', '스레드2', '스레드3']
반환: 스레드1 | 스레드 풀: ['스레드2', '스레드3']
획득: 스레드4 | 스레드 풀: ['스레드2', '스레드3', '스레드4']
반환: 스레드2 | 스레드 풀: ['스레드3', '스레드4']
획득: 스레드5 | 스레드 풀: ['스레드3', '스레드4', '스레드5']
반환: 스레드3 | 스레드 풀: ['스레드4', '스레드5']
획득: 스레드6 | 스레드 풀: ['스레드4', '스레드5', '스레드6']
반환: 스레드4 | 스레드 풀: ['스레드5', '스레드6']
획득: 스레드7 | 스레드 풀: ['스레드5', '스레드6', '스레드7']
반환: 스레드5 | 스레드 풀: ['스레드6', '스레드7']
획득: 스레드8 | 스레드 풀: ['스레드6', '스레드7', '스레드8']
반환: 스레드6 | 스레드 풀: ['스레드7', '스레드8']
획득: 스레드9 | 스레드 풀: ['스레드7', '스레드8', '스레드9']
반환: 스레드7 | 스레드 풀: ['스레드8', '스레드9']
반환: 스레드8 | 스레드 풀: ['스레드9']
반환: 스레드9 | 스레드 풀: []


## 데드락과 스핀락
**데드락의 네 가지 조건(4가지 동시에 만족할 때 발생한다)**
- 상호 배제 : 자원은 한 번에 한 프로세스(혹은 스레드)만 사용할 수 있다.
- 점유와 대기 : 한 프로세스가 자원을 가지고 있는 상태에서, 다른 프로세스가 쓰는 자원의 반납을 기다린다.
- 비선점 : 다른 프로세스가 이미 점유한 자원을 강제로 뺏어오지 못한다.
- 순환 대기 : 프로세스 A,B,C가 있다고 가정할 때 A는 B가 점유한 자원을, B는 C가 점유한 자원을, C는 A가 점유한 자원을 대기하는 상태다.

## 스레딩에 대한 구글 파이썬 스타일 가이드

In [7]:
import threading

def consumer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 대기".format(name))
        cond.wait()
        print("{0} 자원 소비".format(name))
        
def producer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 자원 생산 후 모든 소비자에게 알림".format(name))
        cond.notifyAll()

In [8]:
condition = threading.Condition()
consumer1 = threading.Thread(name='소비자1', target=consumer, args=(condition,))
consumer2 = threading.Thread(name='소비자2', target=consumer, args=(condition,))
producer = threading.Thread(name='생산자', target=producer, args=(condition,))

consumer1.start()
consumer2.start()
producer.start()

소비자1 시작
소비자1 대기
소비자2 시작
소비자2 대기
생산자 시작
생산자 자원 생산 후 모든 소비자에게 알림
소비자1 자원 소비
소비자2 자원 소비


# 프로파일링
- 읽기 전용 데이터는 리스트 대신 튜플을 사용한다.
- 반복문에서 항목이 많은 리스트나 튜플 대신 제너레이터를 사용하여 순회한다.
- 문자열을 연결할 때 + 연산자로 문자열을 연결하는 대신, 리스트에 문자열을 추가한 후, 마지막에 리스트의 항목을 모두 하나로 연결한다.

## cProfile 모듈
호출 시간에 대한 세부 분석을 제공한다 (병목 현상을 찾을 때 유용)

In [10]:
import cProfile
import time

def sleep():
    time.sleep(5)
    
def hello_world():
    print('Hello World!')
    
def main():
    sleep()
    hello_world()

In [11]:
cProfile.run('main()')

Hello World!
         42 function calls in 5.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    5.000    5.000 <ipython-input-10-14a0ff266150>:10(main)
        1    0.000    0.000    5.000    5.000 <ipython-input-10-14a0ff266150>:4(sleep)
        1    0.000    0.000    0.000    0.000 <ipython-input-10-14a0ff266150>:7(hello_world)
        1    0.000    0.000    5.000    5.000 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 iostream.py:197(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:309(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:322(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:384(write)
        3    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        3    0.000    0.000    0.000    0.000 socket.py:342(send)
        3    0.000    0.000    0.000    0.000 threading.py:1062(

## timeit 모듈

In [12]:
import timeit
timeit.timeit("x = 2 + 2")

0.013816

In [13]:
timeit.timeit("x = sum(range(10))")

0.39754860000000036

In [14]:
import time

def sumOfN2(n):
    start = time.time()
    theSum = 0
    for i in range(1, n+1):
        theSum = theSum + i
    end = time.time()
    return theSum, end-start

In [15]:
n = 5
print('총 합계: %d\t 시간: %10.7f초' % sumOfN2(n))

총 합계: 15	 시간:  0.0000000초


In [16]:
n = 200
print('총 합계: %d\t 시간: %10.7f초' % sumOfN2(n))

총 합계: 20100	 시간:  0.0000000초


# 단위 테스트
<br/>

**용어**
- 테스트 픽스처 : 테스트 설정을 위한 코드 (예 : 테스트용 입력 파일을 만들었다 삭제하는 코드)
- 테스트 케이스 : 테스트의 기본 단위
- 테스트 스위트 : unittest.TestCase의 하위 클래스에 의해 생성된 테스트 케이스 집합. 각 테스트 케이스의 메서드 이름은 test로 시작한다.
- 테스트 러너 : 하나 이상의 테스트 스위트를 실행하는 객체

## doctest

In [17]:
def factorial(n):
    import math
    if not n >= 0:
        raise ValueError('n must be >= 0')
    if math.floor(n) != n:
        raise ValueError("n must be exact integer")
    if n+1 == n:
        raise OverflowError("n too large")
    result = 1
    factor = 2
    while factor <= n:
        result *= factor
        factor += 1
    return result

In [19]:
import doctest
import unittest
import 파이썬 파일

suite = unittest.TestSuite()
suite.addTest(doctest.DocTestSuite(파이썬 파일))
runner = unittest.TextTestRunner()
print(runner.run(suite))

TypeError: Expected a module, string, or None